Vyron-Georgios Anemogiannis 1115202000008  \
Georgios-Alexandros Vasilakopoulos 1115202000018

Assignment Written Locally

In [2]:
import pandas as pd
import numpy as np
import sklearn
import math
import matplotlib.pyplot as plt
import wordcloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
import pickle
from math import ceil
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, RocCurveDisplay, roc_curve, auc
from sklearn.metrics import f1_score 
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier


from sklearn import svm
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

import joblib


[nltk_data] Downloading package stopwords to /home/byron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/byron/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load csv
df = pd.read_csv('books_1.Best_Books_Ever.csv')

def convert_to_list(row):
    return eval(row)

df['genres'] = df['genres'].apply(convert_to_list)
df['genreSingle'] = df['genres'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
# keep bookId, description, genreSingle columns
df = df[['bookId', 'description', 'genreSingle']]
# drop rows with NaN
df = df.dropna()

# keep only the rows that have one of the top 10 gerneSingle values
top10 = df['genreSingle'].value_counts().head(10).index.tolist()
df = df[df['genreSingle'].isin(top10)]
df

,bookId,description,genreSingle
0,2767052-the-hunger-games,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,Young Adult
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,There is a door at the end of a silent corrido...,Fantasy
2,2657.To_Kill_a_Mockingbird,The unforgettable novel of a childhood in a sl...,Classics
3,1885.Pride_and_Prejudice,Alternate cover edition of ISBN 9780679783268S...,Classics
4,41865.Twilight,About three things I was absolutely positive.\...,Young Adult
...,...,...,...
52471,12464361-elemental,When seventeen-year-old Kara Nightingale is su...,Fantasy
52472,12395883-unbelievable,Lilah Lopez Dreser's in town to take care of u...,Romance
52474,11836711-anasazi,"'Anasazi', sequel to 'The Thirteenth Chime' by...",Mystery
52475,10815662-marked,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,Fantasy


In [3]:
df['description'] = [r.lower() for r in df['description'] ]   #making everything lowercase
df['description'] = df['description'].apply(lambda r: re.sub(r'[^a-z]', ' ', r))   #only keeping letters
df['description'] = df['description'].apply(lambda r: ' '.join([word for word in r.split() if word not in (stop_words)]))


In [4]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

In [5]:
# enumerate the 10 genres and keep the enumeration in the df as well as a mapping
genre_mapping = {}
for i, genre in enumerate(top10):
	genre_mapping[i] = genre
	df.loc[df['genreSingle'] == genre, 'genreSingle'] = i
    
df

,bookId,description,genreSingle
0,2767052-the-hunger-games,winning means fame fortune losing means certai...,3
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,door end silent corridor haunting harry pottte...,1
2,2657.To_Kill_a_Mockingbird,unforgettable novel childhood sleepy southern ...,8
3,1885.Pride_and_Prejudice,alternate cover edition isbn since immediate s...,8
4,41865.Twilight,three things absolutely positive first edward ...,3
...,...,...,...
52471,12464361-elemental,seventeen year old kara nightingale suddenly s...,1
52472,12395883-unbelievable,lilah lopez dreser town take care unfinished f...,2
52474,11836711-anasazi,anasazi sequel thirteenth chime emma michaels ...,6
52475,10815662-marked,readers favorite awards winner sixteen year ol...,1


In [6]:
def vectorize(df, w2v):
	X = list()
	Y = list()
	gernes = df['genreSingle'].values


	for (i, review) in enumerate(df["description"].values):
		wvectors = list()
		for word in review.split(" "):
			if word in w2v.keys(): # Keeping only words in the vocabulary and using theur value
				wvectors.append(list(w2v[word]))
		if len(wvectors) > 0:
			X.append(np.mean(wvectors, axis=0)) #we keep the mean 
			Y.append(gernes[i])
	return (X,Y)



w2v = {}

with open('glove.6B.300d.txt', encoding="utf8") as fp:
	for lines in fp:
		varray = lines.split(" ")
		w2v[varray[0]] = np.asarray(varray[1:], dtype='float32')

(X, Y) = vectorize(df, w2v)



In [7]:
with open('X.pkl', 'wb') as f:
    pickle.dump(X, f)
with open('Y.pkl', 'wb') as f:
    pickle.dump(Y, f)
with open('genre_mapping.pkl', 'wb') as f:
    pickle.dump(genre_mapping, f)

### Cleaned

In [10]:
with open('X.pkl', 'rb') as f:
    X = pickle.load(f)
with open('Y.pkl', 'rb') as f:
    Y = pickle.load(f)
with open('genre_mapping.pkl', 'rb') as f:
    genre_mapping = pickle.load(f)

In [11]:
Xtrain, Xtest , Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=69)

# Naive Bayes

In [15]:
gnb = GaussianNB()
gnb.fit(Xtrain, Ytrain)
Ypred = gnb.predict(Xtest)
print(classification_report(Ytest, Ypred, target_names=genre_mapping.values()))

scores = cross_val_score(gnb, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
f1 = f1_score(Ytest, Ypred, average='macro')
print('F1 score: %f' % f1)


                    precision    recall  f1-score   support

           Fiction       0.51      0.04      0.07      1154
           Fantasy       0.82      0.33      0.47      1111
           Romance       0.40      0.01      0.02       630
       Young Adult       0.23      0.78      0.36       616
        Nonfiction       0.46      0.42      0.44       507
Historical Fiction       0.42      0.37      0.39       442
           Mystery       0.33      0.78      0.46       421
   Science Fiction       0.45      0.55      0.49       281
          Classics       0.12      0.31      0.18       227
           History       0.39      0.40      0.39       182

          accuracy                           0.34      5571
         macro avg       0.41      0.40      0.33      5571
      weighted avg       0.48      0.34      0.30      5571

Accuracy: 0.32 (+/- 0.01)
F1 score: 0.327082


# Support vector Machines

In [4]:
def objective(trial):
	kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
	c = trial.suggest_float('c', 1e-2, 10, log=True)
	gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    
	clf = svm.SVC(kernel=kernel, C=c, gamma=gamma)
	clf.fit(Xtrain, Ytrain)
	Ypred = clf.predict(Xtest)
	f1 = f1_score(Ytest, Ypred, average='macro')
	return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(study.best_params)

# {'kernel': 'rbf', 'c': 3.9153433857642326, 'gamma': 'scale'}


[I 2023-05-07 11:26:49,376] A new study created in memory with name: no-name-3079ce67-c799-45a7-9b0a-23186dd711a4
[I 2023-05-07 11:29:38,330] Trial 0 finished with value: 0.3440596910620287 and parameters: {'kernel': 'rbf', 'c': 0.04111714115664755, 'gamma': 'scale'}. Best is trial 0 with value: 0.3440596910620287.
[I 2023-05-07 11:31:03,782] Trial 1 finished with value: 0.6217920198414303 and parameters: {'kernel': 'linear', 'c': 3.141043061182334, 'gamma': 'auto'}. Best is trial 1 with value: 0.6217920198414303.
[I 2023-05-07 11:34:16,169] Trial 2 finished with value: 0.03431970260223048 and parameters: {'kernel': 'rbf', 'c': 0.03499617688808586, 'gamma': 'auto'}. Best is trial 1 with value: 0.6217920198414303.
[I 2023-05-07 11:35:43,084] Trial 3 finished with value: 0.6154665905381511 and parameters: {'kernel': 'linear', 'c': 0.39383942450460424, 'gamma': 'auto'}. Best is trial 1 with value: 0.6217920198414303.
[I 2023-05-07 11:38:25,770] Trial 4 finished with value: 0.5078477515777

{'kernel': 'rbf', 'c': 3.9153433857642326, 'gamma': 'scale'}


In [18]:
joblib.dump(study, "study.pkl")

['study.pkl']

In [2]:
study = joblib.load("study.pkl")

In [3]:
plot_contour(study)

In [4]:
plot_edf(study)

In [5]:
plot_optimization_history(study)

In [6]:
plot_parallel_coordinate(study)

In [7]:
plot_param_importances(study)

In [8]:
plot_slice(study)

In [13]:
clf = svm.SVC(kernel='rbf', C=3.9153433857642326, gamma='scale')
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)
print(classification_report(Ytest, Ypred, target_names=genre_mapping.values()))

scores = cross_val_score(clf, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
f1 = f1_score(Ytest, Ypred, average='macro')
print('F1 score: %f' % f1)

                    precision    recall  f1-score   support

           Fiction       0.57      0.72      0.64      1154
           Fantasy       0.77      0.79      0.78      1111
           Romance       0.69      0.69      0.69       630
       Young Adult       0.61      0.59      0.60       616
        Nonfiction       0.78      0.69      0.74       507
Historical Fiction       0.68      0.60      0.64       442
           Mystery       0.77      0.69      0.73       421
   Science Fiction       0.77      0.64      0.70       281
          Classics       0.62      0.32      0.42       227
           History       0.70      0.62      0.66       182

          accuracy                           0.68      5571
         macro avg       0.70      0.63      0.66      5571
      weighted avg       0.69      0.68      0.68      5571

Accuracy: 0.66 (+/- 0.04)
F1 score: 0.657300


In [14]:
# classify using random forests from sklearn
forest = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=420)
forest.fit(Xtrain, Ytrain)
Ypred = forest.predict(Xtest)
print(classification_report(Ytest, Ypred, target_names=genre_mapping.values()))

scores = cross_val_score(forest, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
f1 = f1_score(Ytest, Ypred, average='macro')
print('F1 score: %f' % f1)

/home/byron/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/byron/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/byron/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



                    precision    recall  f1-score   support

           Fiction       0.28      0.85      0.43      1154
           Fantasy       0.49      0.77      0.60      1111
           Romance       0.49      0.29      0.36       630
       Young Adult       0.00      0.00      0.00       616
        Nonfiction       0.00      0.00      0.00       507
Historical Fiction       0.00      0.00      0.00       442
           Mystery       0.00      0.00      0.00       421
   Science Fiction       0.00      0.00      0.00       281
          Classics       0.00      0.00      0.00       227
           History       0.00      0.00      0.00       182

          accuracy                           0.36      5571
         macro avg       0.13      0.19      0.14      5571
      weighted avg       0.21      0.36      0.25      5571

Accuracy: 0.36 (+/- 0.02)
F1 score: 0.139406


In [3]:

# create and print table with all the results
results = pd.DataFrame(columns=['model', 'accuracy', 'f1'])
results.loc[0] = ['Naive Bayes', 0.32, 0.327082]
results.loc[1] = ['SVM', 0.66, 0.657300]
results.loc[2] = ['Random Forest', 0.36, 0.139406]
results

,model,accuracy,f1
0,Naive Bayes,0.32,0.327082
1,SVM,0.66,0.657300
2,Random Forest,0.36,0.139406
